In [2]:
#take tags only from test5 database

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter


df = pd.read_csv('train.csv')

In [163]:
test_data = pd.read_csv('test5_all.csv')

In [164]:
# delete all data with empty tags from test data
test_data.dropna(subset=['photoTags'], inplace=True)

In [145]:
#take tags from the column 'photoTags'
tags = test_data.loc[:,'photoTags']

In [147]:
# join them to list
text = ' '.join(test_data['photoTags'])
text_list = text.split(' ')

In [176]:
vectorizer = CountVectorizer()
vectorizer.fit_transform(tags)

<216987x185595 sparse matrix of type '<class 'numpy.int64'>'
	with 1762366 stored elements in Compressed Sparse Row format>

In [177]:
print(len(vectorizer.vocabulary_))

185595


In [178]:
# save the corpus into the file
import csv

dt = vectorizer.vocabulary_


d = pd.DataFrame.from_dict(dt, orient='index', dtype=None)

d.to_csv("corpus1.csv", encoding="utf-8", header=['id'])

In [221]:
#LOAD DATA
train = pd.read_csv("corpus1.csv");

#CLEAN DATA
train['words'] = train['words'].str.replace("\d+", '')
train['words'] = train['words'].str.replace("sec", '')
train['words'] = train['words'].str.replace("nabkv", '')

#train['words'] = train['words'].str.replace("^[0-9]*$", '')
#s = re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", s)
train['words'].replace('', np.nan, inplace=True)
train = train.dropna(subset=['words'])


In [229]:
train = train[train['words'].str.len() > 3]


In [230]:
train.to_csv("corpus.csv", encoding="utf-8")

In [231]:
print(len(train))

151710
